In [28]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import numpy as np
import pandas as pd
 

In [29]:
#laod trained model, scalar picke , onehot 
model = load_model('model.h5')

with open("label_encoder_gender.pkl",'rb') as file:
  label_encoder_gender = pickle.load(file)

with open("onehot_encoder_geo.pkl",'rb')as file:
  one_hot_encoder_geo = pickle.load(file)

with open ('scalar.pkl','rb') as file :
  scalar = pickle.load(file)
  

In [48]:
input_data = {
'CreditScore' : 600,  ## here values are scalar , like 500, france , if it was vector than it would be like [600] so we have to say how many rows to make in data frame while converting dict to dataframe 
'Geography' : 'France',
'Gender': 'Male',
'Age': 8,
'Tenure': 3, 
'Balance' : 60000,
'NumOfProducts': 2, 
'HasCrCard':1,
'IsActiveMember': 1, 
'EstimatedSalary':50000

 }
input_data['Geography']


'France'

In [49]:
geo_encoded = one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray() ##retruns  array
geo_encoded = pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded

C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [50]:
inputDf = pd.DataFrame(input_data, index = [0]) ##index because values in dict are scalars |

In [51]:
input_data = pd.concat([inputDf.drop(["Geography"],axis=1),geo_encoded],axis = 1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,8,3,60000,2,1,1,50000,1.0,0.0,0.0


In [52]:
#encode categorical variables  
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,8,3,60000,2,1,1,50000,1.0,0.0,0.0


In [53]:
input_scaled = scalar.transform(input_data)
input_scaled

array([[-0.53598516,  0.91324755, -2.93752437, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [54]:
prediction = model.predict(input_scaled)
prediction_probab = prediction[0][0]
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([[0.00768411]], dtype=float32)

In [55]:
if (prediction_probab >0.5):
  print("the employee is likely to churn")
else:
  print("the employee is not likely to churn")

the employee is not likely to churn


In [56]:
one_hot_encoder_geo.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)